### 检查&安装模块

In [ ]:
# !pip list
# !pip show pymongo
# !pip install package_name==version #安装工具包
# !pip install package_name --upgrade #更新工具包
!pip install fake_useragent

### 导入数据库

In [ ]:
from mongo_cli import db_dev as db
hsu = db["hsu"]
res = hsu.find({'status':0}).sort("_id",-1).limit(3)
sid_list = [x.get('sid') for x in res]
print(sid_list)

### 路由分析

In [ ]:
# 结构分析
"""
https://movie.douban.com/j/search_subjects?type=tv&tag=韩剧&sort=rank&page_limit=20&page_start=20
https://movie.douban.com/j/search_subjects?type=tv&tag=英剧&sort=rank&page_limit=20&page_start=400
tv 热门/美剧/英剧/韩剧/日剧/国产剧/港剧/日本动画/综艺/纪录片
https://movie.douban.com/j/search_subjects?type=movie&tag=%E6%AC%A7%E7%BE%8E&sort=rank&page_limit=20&page_start=0
movie 热门  最新  经典  可播放  豆瓣高分  冷门佳片  华语  欧美  韩国  日本  动作 喜剧  爱情  科幻  悬疑  恐怖  文艺

{
  "subjects": [
    {
      "rate": "9.7",
      "cover_x": 500,
      "title": "大明王朝1566",
      "url": "https://movie.douban.com/subject/2210001/",
      "playable": true,
      "cover": "https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2444453078.webp",
      "id": "2210001",
      "cover_y": 700,
      "is_new": false
    },
"""

### 导包

In [73]:
import re
import requests
import json
import random
import traceback
from lxml import etree

from fake_useragent import UserAgent
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

### 公共模块

In [72]:
PROXIES_POOL = [
    'squidsz01.dianhua.cn',
    'squidsz02.dianhua.cn',
    'squidsz03.dianhua.cn',
    'squidsz04.dianhua.cn',
    'squidsz05.dianhua.cn',
    'squidsz06.dianhua.cn',
    'squidsz07.dianhua.cn',
    'squidsz08.dianhua.cn',
    'squidsz09.dianhua.cn',
    'squidsz10.dianhua.cn',
]

max_tries = 3
time_out = 20

def random_proxies():
    proxies = None
    if PROXIES_POOL:
        try:
            IP = random.choice(PROXIES_POOL)
            proxies = {
                "http": "http://{ip}:{port}".format(ip=IP, port=8080),
                "https": "http://{ip}:{port}".format(ip=IP, port=8080)
            }
        except:
            return proxies
    return proxies

def deal_request(method, url, headers=None, data=None, params=None, cookies=None):
    session = requests.Session()
    session.headers.update({"User-Agent":UserAgent().chrome})
    session.proxies.update(random_proxies())
#     print(session.proxies)
    for _ in range(max_tries):
        try:
            if method == "post":
                return session.post(url, headers=headers, data=data, params=params,
                                    #proxies=proxies, 
                                    cookies=cookies, 
                                    allow_redirects=True, verify=False, timeout=time_out)
            elif method == "get":
                return session.get(url, headers=headers, params=params,
                                   #proxies=proxies,
                                   stream=True,
                                   cookies=cookies, 
                                   allow_redirects=True, verify=False, timeout=time_out)
        except (requests.ConnectionError, requests.Timeout):
            error = traceback.format_exc()
            msg = u"运营商官网错误,请求失败:{}".format(error)
            print(msg)
            continue
        except:
            error = traceback.format_exc()
            msg = u"遇到未知错误,请求失败:{}".format(error)
            msg_dict = {
                'msg':msg,
                'method':method,
                'url': url,
                'headers': headers,
                'data': data,
                'params': params,
                'proxies': proxies
            }
            raise msg_dict
    else:
        msg_dict = {
            'msg':msg,
            'method':method,
            'url': url,
            'headers': headers,
            'data': data,
            'params': params,
            'proxies': self.proxies
        }
        raise msg_dict

def post(url, headers=None, data=None, params=None, cookies=None):
    return deal_request('post', url, headers=headers, data=data, params=params, cookies=cookies)

def get(url, headers=None, params=None, cookies=None):
    return deal_request('get', url, headers=headers, params=params, cookies=cookies)


### 检查代理&UA使用情况

In [79]:
url = "http://rate.taobao.com/feedRateList.htm?auctionNumId=574096941560&currentPageNum=1&pageSize=20/"
resp = get(url)
print(resp.text) 
# 查看请求头
print(resp.request.headers)
# 查看对方ip
print(resp.raw._connection.sock.socket.getsockname())
# 查看自己使用的ip
resp.json()



{"rgv587_flag":"sm","url":"https://login.taobao.com/member/login.jhtml?style=mini&from=sm&full_redirect=false&redirectURL=http%3a%2f%2frate.taobao.com/feedRateList.htm/_____tmd_____/punish%3fx5secdata=5e0c8e1365474455070961b803bd560607b52cabf5960afff39b64ce58073f7844a8dd94c89edef01763e7bda2153e72b3cd95a333f87ac5f6b549e857d928307bf3fd8381b0cf846dca6234fbaf10bc77b99fcb1ba2eeec835ece334a00d33d8de558990b51a94454a308719ae637b507e8eed4b79d54b22ab681b565fd4455fd18164a32ef91d6c13e43760837bb8b23d59fe344f7687f024908904d745f44ba626cc1d2f3a369547dcfe46442163f1331473ca903699d7ed69bb307a0ad9618c3c748ed00d72f6d6f0b5c29c41377264d7a167c0b37cbca29241a2ab5c9070aa108b6b72dbeca910999ec587efd9ad10328165050f1d1a68aedfc913e2c635fb2baefb62c5a97c3f01215fddfbd8074416dfefbeef8a3a6988d4239a9bf2d77a24be6ec432e06e961436700320a89ac62b8c07ac9734d6799e4f8b11e11a59dab80f89293688707c3aec5d53b80ed975b013c6fad960df313a9c9aea91878a96e1895f9d9f1344cb5c80a4a6c72581e8553dc1f220d787060963ca76c593ae16852c052af527827fdb8d24e0c8

AttributeError: 'NoneType' object has no attribute 'sock'

In [ ]:
def get_a_page(url):
    resp = get(url).json().get("subjects", [])
    if not resp:
        return []
    return [{'title' : item.get('title'),
             'rate'  : item.get('rate'),
             'url'   : item.get('url'),
             'cover' : item.get('cover'),
             'id'    : item.get('id') } for item in resp]
    
def add_items(items):
    url = items.get('url')
    resp = get(url)
#     print(resp.text)
#     print(re.match('<script type="application/ld+json">(.*?)</script>',resp.text))
    et = etree.HTML(resp.text)
#     print(et.xpath('//*[@id="info"]/span[2]/span[@class="attrs"]/a/text()'))
    data = {
        'director':et.xpath('//*[@id="info"]/span[1]/span[@class="attrs"]/a/text()'),
        'scenarist':et.xpath('//*[@id="info"]/span[2]/span[@class="attrs"]/a/text()'),
        'actor':et.xpath('//*[@id="info"]/span[3]/span[@class="attrs"]/a/text()'),
    }
    return dict(items,**data)

url = "https://movie.douban.com/j/search_subjects?type=tv&tag=韩剧&sort=rank&page_limit=20&page_start=20"
resp = get_a_page(url)
data = [add_items(i) for i in resp]

In [59]:
print(data)

NameError: name 'data' is not defined

### pandas隐藏BUG
> 当一组元素的值的数量是一致的时候，自动将列表转化为元组

In [63]:
import pandas as pd
a = pd.DataFrame()
a['z'] = [1,2, 1, 2,1]
a['p'] = ([],[1],[1],[1],[1])
a['v'] = ([1,2],[1],[1],[1],[1])
a['b'] = (['1'],[1],[1],['1'],[1])
a['c'] = ([],[],[],[],[])
a['d'] = ([1,2],[1,2],[1,2],[1,2],[1,2])
a['q'] = [[1,2],[1,2],[1,2],[1,2],[1,2]]
a

,z,p,v,b,c,d,q
0,1,[],"[1, 2]","(1,)",(),"(1, 2)","[1, 2]"
1,2,[1],[1],"(1,)",(),"(1, 2)","[1, 2]"
2,1,[1],[1],"(1,)",(),"(1, 2)","[1, 2]"
3,2,[1],[1],"(1,)",(),"(1, 2)","[1, 2]"
4,1,[1],[1],"(1,)",(),"(1, 2)","[1, 2]"


In [69]:
a = [1,2,3]
b = [4,5,6]
c = [4,5,6,7,8]
print(list(zip(a,b)))         # 打包为元组的列表
print(list(zip(a,c)))         # 元素个数与最短的列表一致
print(list(zip(*zip(a,b))))   # 与 zip 相反，*zip 可理解为解压，返回二维矩阵式

[(1, 4), (2, 5), (3, 6)]
[(1, 4), (2, 5), (3, 6)]
[(1, 2, 3), (4, 5, 6)]


In [70]:
import requests
from fake_useragent import UserAgent
requests.get('',headers={"User-Agent":UserAgent().chrome}).text

'\r\n\r\n{"rgv587_flag":"sm","url":"https://login.taobao.com/member/login.jhtml?style=mini&from=sm&full_redirect=false&redirectURL=http%3a%2f%2frate.taobao.com/feedRateList.htm/_____tmd_____/punish%3fx5secdata=5e0c8e1365474455070961b803bd560607b52cabf5960afff39b64ce58073f7844a8dd94c89edef01763e7bda2153e72b3cd95a333f87ac5f6b549e857d928307bf3fd8381b0cf846dca6234fbaf10bc77b99fcb1ba2eeec835ece334a00d33d8de558990b51a94454a308719ae637b59d74b781c9f437af72e3284ec4e64f576f873d3fe064fae5559369674b93155c5f7889dc13a1a50660f5d6ddf442abaaba626cc1d2f3a369547dcfe46442163f1331473ca903699d7ed69bb307a0ad9618c3c748ed00d72f6d6f0b5c29c41377264d7a167c0b37cbca29241a2ab5c9070aa108b6b72dbeca910999ec587efd9ad10328165050f1d1a68aedfc913e2c635fb2baefb62c5a97c3f01215fddfbd8074416dfefbeef8a3a6988d4239a9bf2d77a24be6ec432e06e961436700320a89ac62b8c07ac9734d6799e4f8b11e11a59dab80f89293688707c3aec5d53b80ed975b013c6fad960df313a9c9aea91878a96e1895f9d9f1344cb5c80a4a6c72581e8553dc1f220d787060963ca76c593ae16852c052af527827fdb8